<a href="https://colab.research.google.com/github/gjwnsdud5076/Jun/blob/master/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# object 예측과 crop

test용 jpg / trained weight 업로드하기

In [2]:
!git clone 'https://github.com/gjwnsdud5076/drug.git'

Cloning into 'drug'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [3]:
!git clone  'https://github.com/ultralytics/yolov5.git'

Cloning into 'yolov5'...
remote: Enumerating objects: 3271, done.
remote: Total 3271 (delta 0), reused 0 (delta 0), pack-reused 3271
Receiving objects: 100% (3271/3271), 6.48 MiB | 34.39 MiB/s, done.
Resolving deltas: 100% (2180/2180), done.


In [4]:
!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies

     |████████████████████████████████| 276kB 13.9MB/s 


detect.py 바꾸기

In [5]:
import os
os.chdir('/content/yolov5')
os.getcwd()

'/content/yolov5'

In [6]:
from google.colab import files
uploaded = files.upload()

Saving detect (1).py to detect (1).py


In [7]:
os.chdir('/content')
os.getcwd()

'/content'

In [8]:
output = !python 'yolov5/detect (1).py' --source "/content/drug/box/*.jpg" --weights '/content/drug/best.pt' --save-txt
print(output)

["Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=True, source='/content/drug/box/*.jpg', update=False, view_img=False, weights=['/content/drug/best.pt'])", 'Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)', '', 'Fusing layers... ', 'Model Summary: 232 layers, 7246518 parameters, 0 gradients, 16.8 GFLOPS', 'image 1/3 /content/drug/box/jiheum (10).jpg: 352x640 3 texts, Done. (0.030s)', 'image 2/3 /content/drug/box/jiheum (11).jpg: 352x640 3 texts, Done. (0.012s)', 'image 3/3 /content/drug/box/jiheum (12).jpg: 352x640 7 texts, Done. (0.010s)', 'Results saved to runs/detect/exp', '3 labels saved to runs/detect/exp/labels', 'Done. (0.244s)']


'Results saved to runs/detect/exp(횟수)', '3 labels saved to runs/detect/exp(횟수)/labels'

crop

In [9]:
path='/content/drug/box/jiheum (10).jpg'
name=path.split("/")[4].split(".")
name=name[0]
print(name)


jiheum (10)


In [12]:
import re
path='/content/runs/detect/exp/labels/jiheum (10).txt'
myfile=open(path,'r')
lines=myfile.readlines()
pattern= "0"

boxes=[]
for line in lines:
  if re.search(pattern,line):
    Cord_Raw=line
    Cord=Cord_Raw.split( )
    boxes.append(Cord)

print(boxes)

[['0', '494', '169', '525', '205'], ['0', '197', '164', '242', '212'], ['0', '531', '169', '562', '206']]


In [26]:
import cv2
img = cv2.imread("/content/drug/box/jiheum (10).jpg")
cnt = 0
for box in boxes:
    a=box[2] 
    b=box[4]
    c=box[1]
    d=box[3]
    crop_img = img[int(a):int(b), int(c):int(d)]
    cropped_img='/content/drug/crop_img/S/'+name+"_"+str(cnt)+".jpg"
    cv2.imwrite(cropped_img,crop_img)
    cnt += 1


In [15]:
!cp '/content/drug/LeNet5_model.py' 'LeNet5_model.py'

약 정보가 담길 곳 drug_info 
글자, 색상, 모양 순

In [17]:
drug_info=[]

# 글자예측

In [31]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from LeNet5_model import LeNet5_model, Config
#import matplotlib.pyplot as pp


if __name__ == "__main__":
    cfg = Config()

    device = torch.device("cpu")  
    print("device:", device)

    # 모델 생성
    model = LeNet5_model()


    if torch.cuda.is_available():
        model = model.to(device)

    model.eval()

    save_path = "/content/drug/epoch_21.pth"
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])


    classes = ('O', 'P', 'Q', 'R', 'S', 'T', 'U')

    transforms_test = transforms.Compose([
        transforms.Resize([42, 42]),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    imgs = ImageFolder('/content/drug/crop_img', transform=transforms_test)
    print("img shape",imgs[0][0].shape)
    print("imgs:", imgs)
    test_loader = DataLoader(imgs, batch_size=1)


    print("test_loader:", test_loader)
    print(test_loader.dataset)
    text = []
    for thisimg, label in test_loader:
        pred = model.forward(thisimg.to(device))
        _, top_pred = torch.topk(pred, k=1, dim=-1)
        top_pred = top_pred.squeeze(dim=1)
        print("--------------------------------------")
        #print("truth:", classes[label])
        print("model prediction:", classes[top_pred])
        text.append(classes[top_pred])

drug_info.append(text)


device: cpu
img shape torch.Size([3, 42, 42])
imgs: Dataset ImageFolder
    Number of datapoints: 3
    Root location: /content/drug/crop_img
    StandardTransform
Transform: Compose(
               Resize(size=[42, 42], interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )
test_loader: <torch.utils.data.dataloader.DataLoader object at 0x7fa5cde959b0>
Dataset ImageFolder
    Number of datapoints: 3
    Root location: /content/drug/crop_img
    StandardTransform
Transform: Compose(
               Resize(size=[42, 42], interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )
--------------------------------------
model prediction: P
--------------------------------------
model prediction: O
--------------------------------------
model prediction: S


# 색상 예측하기

In [41]:
    #src = cv2.imread('black.JPG', cv2.IMREAD_COLOR)
    src = cv2.imread(cropped_img, cv2.IMREAD_COLOR)
    hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    H=h[2][2]*2
    S=s[2][2]/255
    V=v[2][2]/255
    print(H, S, V)
    W = (1 - S)*V*100 #HSV to HWB
    B = (1 - V)*100

    print(H, W, B)

    if (90<=W<=100)|(W+B>=95):
        if  85<=B<=100:
         drug_info.append("black")
        else:
         drug_info.append("white")
    # W+B>=95:
    #   return "gray"
    elif (0<=H<13)|(346<=H<=360):
        drug_info.append("red")
    elif 13<=H<28:
        drug_info.append("orange")
    elif 28<=H<63:
        drug_info.append("yellow")
    elif 63<=H<163:
        if B>=30:
            drug_info.append("green")
        else:
            drug_info.append("연두색")
    elif 163<=H<172:
        drug_info.append("청록색")
    elif 172<=H<260:
        if B>=41:
            drug_info.append("남색")
        else:
            drug_info.append("blue")
    elif 260<=H<286:
        drug_info.append("purple")
    elif 286<=H<346:
        if B>=17:
            drug_info.append("자주색")
        else:
            drug_info.append("pink")
    else:
        drug_info.append("error")


354 0.3568627450980392 0.10980392156862745
354 7.061899269511726 89.01960784313725


In [42]:
print(drug_info)

[['P', 'O', 'S'], 'white', 'white', 'white', 'black']


# 모양 예측하기

In [46]:
!mkdir shape
!cp  '/content/drug/box/jiheum (10).jpg' 'shape/jiheum (10).jpg'

mkdir: cannot create directory ‘shape’: File exists
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `cp str(img) 'shape/jiheum (10).jpg''


In [61]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from LeNet5_model_shape import LeNet5_model, Config
import cv2
from os import listdir
from os.path import isfile, join
import numpy

if __name__ == "__main__":
    cfg = Config()

    device = torch.device("cpu")
    print("device:", device)

    # 모델 생성
    model = LeNet5_model()

    model.eval()

    save_path = "/content/drug/epoch_36.pth"
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])


    classes = ('circle', 'diamond', 'hexagon6', 'number8', 'octagon', 'oval', 'ovlong', 'pentagon5', 'rectangle', 'triangle')

    transforms_test = transforms.Compose([
        transforms.Resize([78,42]),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])


    imgs = ImageFolder('/content/shape', transform=transforms_test)
    print("img shape",imgs[0][0].shape)
    print("imgs:", imgs)
    test_loader = DataLoader(imgs, batch_size=1)


    print("test_loader:", test_loader)
    print(test_loader.dataset)

    for thisimg, label in test_loader:
        pred = model.forward(thisimg.to(device))
        _, top_pred = torch.topk(pred, k=1, dim=-1)
        top_pred = top_pred.squeeze(dim=1)
        print("--------------------------------------")
        print("truth:", classes[label])
        print("model prediction:", classes[top_pred])
        drug_info.append(classes[top_pred])



device: cpu
img shape torch.Size([3, 78, 42])
imgs: Dataset ImageFolder
    Number of datapoints: 1
    Root location: /content/shape
    StandardTransform
Transform: Compose(
               Resize(size=[78, 42], interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )
test_loader: <torch.utils.data.dataloader.DataLoader object at 0x7fa5cd3e0588>
Dataset ImageFolder
    Number of datapoints: 1
    Root location: /content/shape
    StandardTransform
Transform: Compose(
               Resize(size=[78, 42], interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )
--------------------------------------
truth: diamond
model prediction: circle


# 결과

In [64]:
print(drug_info)

[['P', 'O', 'S'], 'white', 'white', 'white', 'black', 'circle']
